In [14]:
import pandas as pd

# Load your XPG TSV (the same one that populates your database)
xpgs = pd.read_csv("/Users/Muthu/Downloads/Gene_Annotation/XPGs_with_PV_enrichment.tsv", sep="\t")

# Load your bipolar disorder gene results
bipolar = pd.read_csv("/Users/Muthu/Documents/XPG_Portal/data/bipolar_gene_results.csv")

# Filter bipolar for p < 0.05

# === 3️⃣ Identify significance in each class ===
ptv_sig = bipolar[bipolar["PTV Fisher p‑val"] < 0.05].copy()
ptv_sig["SigType"] = "PTV"

missense_sig = bipolar[bipolar["Damaging Missense Fisher p‑val"] < 0.05].copy()
missense_sig["SigType"] = "Missense"

# Combine and handle duplicates
combined = pd.concat([ptv_sig, missense_sig])

# === 3️⃣ Collapse to single row per gene ===
def summarize_sig(group):
    sig_types = set(group["SigType"])
    sig_type = "Both" if len(sig_types) > 1 else list(sig_types)[0]
    ptv_p = group["PTV Fisher p‑val"].min() if "PTV Fisher p‑val" in group else None
    missense_p = group["Damaging Missense Fisher p‑val"].min() if "Damaging Missense Fisher p‑val" in group else None
    return pd.Series({"SigType": sig_type, "PTV_pval": ptv_p, "Missense_pval": missense_p})

combined_summary = combined.groupby("Gene").apply(summarize_sig).reset_index()

# === 4️⃣ Merge with XPGs ===
overlap = pd.merge(xpgs, combined_summary, left_on="Human_ENSEMBL", right_on="Gene", how="inner")

# === 5️⃣ Select and rename columns ===
final_overlap = overlap[["gene", "Human_ENSEMBL", "SigType", "PTV_pval", "Missense_pval"]]

# === 6️⃣ Save the output ===
output_path = "/Users/Muthu/Documents/XPG_Portal/data/XPGs_in_bipolar_overlap_with_pvals.tsv"
final_overlap.to_csv(output_path, sep="\t", index=False)

print(f"✅ Saved {len(final_overlap)} overlapping XPGs to {output_path}")
final_overlap.head(10)


✅ Saved 64 overlapping XPGs to /Users/Muthu/Documents/XPG_Portal/data/XPGs_in_bipolar_overlap_with_pvals.tsv


/var/folders/d6/jt_cxdls1_542cgj4mcnc1s00000gn/T/ipykernel_42425/2520645790.py:29: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  combined_summary = combined.groupby("Gene").apply(summarize_sig).reset_index()


,gene,Human_ENSEMBL,SigType,PTV_pval,Missense_pval
0,Crocc,ENSG00000058453,PTV,0.043581,0.586150
1,Grm7,ENSG00000196277,Missense,0.366580,0.048984
2,Hectd2,ENSG00000165338,PTV,0.006912,0.241440
3,Il16,ENSG00000172349,Missense,0.058286,0.011418
4,Mast4,ENSG00000069020,Missense,0.186550,0.036778
5,Syde2,ENSG00000097096,Both,0.019490,0.019342
6,Akap11,ENSG00000023516,PTV,0.000012,0.287680
7,Kdm5b,ENSG00000117139,Both,0.011347,0.018432
8,Nek1,ENSG00000137601,PTV,0.017455,0.094120
9,Scn3a,ENSG00000153253,PTV,0.019490,0.573010


In [ ]:
# Query all XPGs from your Django DB
xpgs = Gene.objects.all().values("gene", "Human_ENSEMBL", "log2FC", "adjP")

# Convert to DataFrame for easy matching
xpg_df = pd.DataFrame(list(xpgs))

# Find overlap by Ensembl ID
overlap = xpg_df[xpg_df["Human_ENSEMBL"].isin(bipolar_genes)]

# Display results
print(overlap[["gene", "Human_ENSEMBL", "log2FC", "adjP"]])


In [19]:
df = pd.read_csv("/Users/Muthu/Downloads/Gene_Annotation/XPGs_with_PV_enrichment.tsv", sep="\t")
df.head()

,gene,log2FC,adjP,ENSEMBL,ENTREZID,Human_ENSEMBL,pathways,tract_SM,tract_AB,tract_PR,...,Drugs_SM,Drugs_AB,Drugs_PR,Drugs_OC,Drugs_Enzyme,Drugs_Unknown,Drugs_Antibody drug conjugate,pv_mean,nonpv_mean,pv_enrichment
0,Ylpm1,1.238886,4.180000e-08,ENSMUSG00000021244,56531.0,ENSG00000119596,DNA metabolic process; regulation of organelle...,NaN,NaN,UniProt Ubiquitination; Database Ubiquitinatio...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.193236,1.090323,0.130124
1,Fbxo46,2.799165,3.110000e-06,ENSMUSG00000050428,243867.0,ENSG00000177051,NaN,NaN,NaN,UniProt Ubiquitination; Database Ubiquitination,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.130435,0.082791,0.655773
2,Usp53,1.571387,3.360000e-06,ENSMUSG00000039701,99526.0,ENSG00000145390,action potential; response to mechanical stimu...,NaN,Human Protein Atlas loc,Database Ubiquitination,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.173913,0.156911,0.148417
3,Dock4,1.545629,3.640000e-06,ENSMUSG00000035954,238130.0,ENSG00000128512,positive regulation of vascular associated smo...,NaN,UniProt loc high conf; GO CC high conf,Database Ubiquitination; Half-life Data,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.280192,4.188409,-0.352622
4,Lima1,3.134084,4.680000e-06,ENSMUSG00000023022,65970.0,ENSG00000050405,regulation of actin polymerization or depolyme...,NaN,GO CC high conf; UniProt loc med conf,UniProt Ubiquitination; Database Ubiquitinatio...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.468599,0.161601,1.535916


In [20]:
import mygene

mg = mygene.MyGeneInfo()

# Query with mouse Ensembl IDs
results = mg.querymany(
    df["ENSEMBL"].dropna().tolist(),
    scopes="ensembl.gene",
    fields="symbol,entrezgene,MGI,uniprot.Swiss-Prot,go.BP.term,go.MF.term,go.CC.term",
    species="mouse"
)


Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed
24 input query terms found dup hits:	[('ENSMUSG00000028527', 2), ('ENSMUSG00000031314', 2), ('ENSMUSG00000071636', 3), ('ENSMUSG000000638
1 input query terms found no hit:	['ENSMUSG00000060530']


In [21]:
go_df = pd.DataFrame(results)
go_df.head()

,query,MGI,_id,_score,entrezgene,go,symbol,uniprot,notfound
0,ENSMUSG00000021244,MGI:1926195,56531,27.608355,56531,{'BP': [{'term': 'negative regulation of trans...,Ylpm1,{'Swiss-Prot': 'Q9R0I7'},NaN
1,ENSMUSG00000050428,MGI:2444918,243867,27.608355,243867,"{'BP': [{'term': 'biological_process'}, {'term...",Fbxo46,{'Swiss-Prot': 'Q8BG80'},NaN
2,ENSMUSG00000039701,MGI:2139607,99526,27.608355,99526,"{'BP': [{'term': 'action potential'}, {'term':...",Usp53,{'Swiss-Prot': 'P15975'},NaN
3,ENSMUSG00000035954,MGI:1918006,238130,27.608355,238130,{'BP': [{'term': 'small GTPase-mediated signal...,Dock4,{'Swiss-Prot': 'P59764'},NaN
4,ENSMUSG00000023022,MGI:1920992,65970,27.608355,65970,"{'BP': [{'term': 'lipid metabolic process'}, {...",Lima1,{'Swiss-Prot': 'Q9ERG0'},NaN


In [22]:
def make_go_id(row):
    if pd.notna(row.get("MGI")):
        return row["MGI"]                # ✅ preferred for mouse
    elif isinstance(row.get("uniprot"), dict) and "Swiss-Prot" in row["uniprot"]:
        return f"UniProtKB:{row['uniprot']['Swiss-Prot']}"
    elif pd.notna(row.get("entrezgene")):
        return f"NCBIGene:{int(row['entrezgene'])}"
    else:
        return None

go_df["Mouse_GO_ID"] = go_df.apply(make_go_id, axis=1)

In [25]:
go_df["Mouse_GO_ID"].value_counts()

Mouse_GO_ID
MGI:1923939    27
MGI:1913354    27
MGI:3036236    27
MGI:1915026     4
MGI:2143322     4
               ..
MGI:1933289     1
MGI:1917436     1
MGI:1921456     1
MGI:1890596     1
MGI:1278340     1
Name: count, Length: 1898, dtype: int64

In [26]:
df_merged = df.merge(go_df[["query", "MGI", "Mouse_GO_ID"]], left_on="ENSEMBL", right_on="query", how="left")

In [27]:
df_merged.head()

,gene,log2FC,adjP,ENSEMBL,ENTREZID,Human_ENSEMBL,pathways,tract_SM,tract_AB,tract_PR,...,Drugs_OC,Drugs_Enzyme,Drugs_Unknown,Drugs_Antibody drug conjugate,pv_mean,nonpv_mean,pv_enrichment,query,MGI,Mouse_GO_ID
0,Ylpm1,1.238886,4.180000e-08,ENSMUSG00000021244,56531.0,ENSG00000119596,DNA metabolic process; regulation of organelle...,NaN,NaN,UniProt Ubiquitination; Database Ubiquitinatio...,...,NaN,NaN,NaN,NaN,1.193236,1.090323,0.130124,ENSMUSG00000021244,MGI:1926195,MGI:1926195
1,Fbxo46,2.799165,3.110000e-06,ENSMUSG00000050428,243867.0,ENSG00000177051,NaN,NaN,NaN,UniProt Ubiquitination; Database Ubiquitination,...,NaN,NaN,NaN,NaN,0.130435,0.082791,0.655773,ENSMUSG00000050428,MGI:2444918,MGI:2444918
2,Usp53,1.571387,3.360000e-06,ENSMUSG00000039701,99526.0,ENSG00000145390,action potential; response to mechanical stimu...,NaN,Human Protein Atlas loc,Database Ubiquitination,...,NaN,NaN,NaN,NaN,0.173913,0.156911,0.148417,ENSMUSG00000039701,MGI:2139607,MGI:2139607
3,Dock4,1.545629,3.640000e-06,ENSMUSG00000035954,238130.0,ENSG00000128512,positive regulation of vascular associated smo...,NaN,UniProt loc high conf; GO CC high conf,Database Ubiquitination; Half-life Data,...,NaN,NaN,NaN,NaN,3.280192,4.188409,-0.352622,ENSMUSG00000035954,MGI:1918006,MGI:1918006
4,Lima1,3.134084,4.680000e-06,ENSMUSG00000023022,65970.0,ENSG00000050405,regulation of actin polymerization or depolyme...,NaN,GO CC high conf; UniProt loc med conf,UniProt Ubiquitination; Database Ubiquitinatio...,...,NaN,NaN,NaN,NaN,0.468599,0.161601,1.535916,ENSMUSG00000023022,MGI:1920992,MGI:1920992


In [29]:
.to_csv("/Users/Muthu/Downloads/Gene_Annotation/XPGs_with_PV_enrichment.tsv", sep="\t")df_merged

In [2]:
import pandas as pd 
xpg = pd.read_csv("/Users/Muthu/Downloads/Gene_Annotation/XPGs_with_PV_enrichment.tsv", sep="\t")
desc = pd.read_csv("/Users/Muthu/Downloads/GeneDescriptions.tsv", sep="\t")

# rename so the key matches
desc = desc.rename(columns={"Gene stable ID": "ENSEMBL"})

# optional: clean descriptions
desc["Gene description"] = desc["Gene description"].str.replace(r"\s*\[Source:.*?\]", "", regex=True)

# merge
merged = xpg.merge(desc, on="ENSEMBL", how="left")

merged.head()


,Unnamed: 0,gene,log2FC,adjP,ENSEMBL,ENTREZID,Human_ENSEMBL,pathways,tract_SM,tract_AB,...,Drugs_Unknown,Drugs_Antibody drug conjugate,pv_mean,nonpv_mean,pv_enrichment,query,MGI,Mouse_GO_ID,Gene name,Gene description
0,0,Ylpm1,1.238886,4.180000e-08,ENSMUSG00000021244,56531.0,ENSG00000119596,DNA metabolic process; regulation of organelle...,NaN,NaN,...,NaN,NaN,1.193236,1.090323,0.130124,ENSMUSG00000021244,MGI:1926195,MGI:1926195,Ylpm1,YLP motif containing 1
1,1,Fbxo46,2.799165,3.110000e-06,ENSMUSG00000050428,243867.0,ENSG00000177051,NaN,NaN,NaN,...,NaN,NaN,0.130435,0.082791,0.655773,ENSMUSG00000050428,MGI:2444918,MGI:2444918,Fbxo46,F-box protein 46
2,2,Usp53,1.571387,3.360000e-06,ENSMUSG00000039701,99526.0,ENSG00000145390,action potential; response to mechanical stimu...,NaN,Human Protein Atlas loc,...,NaN,NaN,0.173913,0.156911,0.148417,ENSMUSG00000039701,MGI:2139607,MGI:2139607,Usp53,ubiquitin specific peptidase 53
3,3,Dock4,1.545629,3.640000e-06,ENSMUSG00000035954,238130.0,ENSG00000128512,positive regulation of vascular associated smo...,NaN,UniProt loc high conf; GO CC high conf,...,NaN,NaN,3.280192,4.188409,-0.352622,ENSMUSG00000035954,MGI:1918006,MGI:1918006,Dock4,dedicator of cytokinesis 4
4,4,Lima1,3.134084,4.680000e-06,ENSMUSG00000023022,65970.0,ENSG00000050405,regulation of actin polymerization or depolyme...,NaN,GO CC high conf; UniProt loc med conf,...,NaN,NaN,0.468599,0.161601,1.535916,ENSMUSG00000023022,MGI:1920992,MGI:1920992,Lima1,LIM domain and actin binding 1


In [5]:
merged.rename(columns={
    'Gene description': 'gene_description'
}, inplace=True)
print(merged.columns)
merged.to_csv("/Users/Muthu/Downloads/Gene_Annotation/XPGs_with_PV_enrichment.tsv", sep="\t")

Index(['Unnamed: 0', 'gene', 'log2FC', 'adjP', 'ENSEMBL', 'ENTREZID',
       'Human_ENSEMBL', 'pathways', 'tract_SM', 'tract_AB', 'tract_PR',
       'tract_OC', 'Drugs_SM', 'Drugs_AB', 'Drugs_PR', 'Drugs_OC',
       'Drugs_Enzyme', 'Drugs_Unknown', 'Drugs_Antibody drug conjugate',
       'pv_mean', 'nonpv_mean', 'pv_enrichment', 'query', 'MGI', 'Mouse_GO_ID',
       'Gene name', 'gene_description'],
      dtype='object')


In [6]:
import pandas as pd

df = pd.read_csv("/Users/Muthu/Downloads/Gene_Annotation/XPGs_with_PV_enrichment.tsv", sep="\t")

print(df.columns.tolist())

['Unnamed: 0.1', 'Unnamed: 0', 'gene', 'log2FC', 'adjP', 'ENSEMBL', 'ENTREZID', 'Human_ENSEMBL', 'pathways', 'tract_SM', 'tract_AB', 'tract_PR', 'tract_OC', 'Drugs_SM', 'Drugs_AB', 'Drugs_PR', 'Drugs_OC', 'Drugs_Enzyme', 'Drugs_Unknown', 'Drugs_Antibody drug conjugate', 'pv_mean', 'nonpv_mean', 'pv_enrichment', 'query', 'MGI', 'Mouse_GO_ID', 'Gene name', 'gene_description']


In [7]:
print(df[['gene', 'gene_description']].head())


     gene                 gene_description
0   Ylpm1           YLP motif containing 1
1  Fbxo46                 F-box protein 46
2   Usp53  ubiquitin specific peptidase 53
3   Dock4       dedicator of cytokinesis 4
4   Lima1   LIM domain and actin binding 1


In [8]:
print([col for col in df.columns if 'gene' in col])


['gene', 'gene_description']
